In [23]:
import pandas as pd
import numpy as np
import math

file_path = 'C:\\Users\\carra\\OneDrive\\Yeonze\\DA_데브코스\\최종프로젝트\\데이터셋\\병합\\'

df_tv = pd.read_csv(file_path + 'tn_travel_여행.csv')
df_tm = pd.read_csv(file_path + 'tn_traveller_master_여행객 Master.csv', encoding='ANSI')
df_acthis = pd.read_csv(file_path + 'tn_activity_his_활동내역.csv', encoding='ANSI')
df_actconhis = pd.read_csv(file_path + 'tn_activity_consume_his_활동소비내역.csv')
df_advconhis = pd.read_csv(file_path + 'tn_adv_consume_his_사전소비내역.csv')
df_tc = pd.read_csv(file_path + 'tn_companion_info_동반자정보.csv')
df_lodconhis = pd.read_csv(file_path + 'tn_lodge_consume_his_숙박소비내역.csv', encoding='ANSI')
df_mov = pd.read_csv(file_path + 'tn_move_his_이동내역.csv')
df_movconhis = pd.read_csv(file_path + 'tn_mvmn_consume_his_이동수단소비내역.csv')
df_visarea = pd.read_csv(file_path + 'tn_visit_area_info_방문지정보2nd.csv')
df_code = pd.read_csv('C:\\Users\\carra\\OneDrive\\Yeonze\\DA_데브코스\\최종프로젝트\\데이터셋\\병합\\tc_codeb_코드B.csv', encoding='ANSI')
df_SGG = pd.read_csv('C:\\Users\\carra\\OneDrive\\Yeonze\\DA_데브코스\\최종프로젝트\\데이터셋\\병합\\tc_sgg_시군구코드.csv', encoding='ANSI')
df_move = pd.read_csv('C:\\Users\\carra\\OneDrive\\Yeonze\\DA_데브코스\\최종프로젝트\\데이터셋\\병합\\Movement.csv')
df_cluster = pd.read_csv('temp_clutster.csv')

### 데이터 전처리

In [24]:
def merge_travel_data(df_tm, df_tv):
    df_tm = df_tm.merge(df_tv[['TRAVELER_ID', 'TRAVEL_ID']], on='TRAVELER_ID', how='left')
    return df_tm
df_tm = merge_travel_data(df_tm, df_tv)

In [25]:
def map_and_merge_travel_purpose(df_tm, df_tv):
    mapping_dict = {
        1: 2, 2: 3, 3: 4, 4: 4, 5: 3, 6: 4, 7: 3, 8: 5,
        9: 4, 10: 5, 11: 3, 12: 4, 13: 6, 21: 5, 22: 4, 23: 5,
        24: 4, 25: 4, 26: 4, 27: 3, 28: 3
    }

    def map_travel_purpose(purpose_str):
        purpose_codes = map(int, purpose_str.split(';'))  # 세미콜론으로 분리 후 각 코드 변환
        mapped_values = [str(mapping_dict.get(code, code)) for code in purpose_codes]  # 매핑 적용
        return ';'.join(mapped_values)  # 세미콜론으로 결합하여 반환

    # TRAVEL_PURPOSE 값에 매핑 함수 적용
    df_tv['TRAVEL_PURPOSE'] = df_tv['TRAVEL_PURPOSE'].apply(map_travel_purpose)

    # df_tm과 df_tv를 TRAVEL_ID 기준으로 병합하여 mapped_TRAVEL_PURPOSE 컬럼 추가
    df_tm = df_tm.merge(df_tv[['TRAVEL_ID', 'TRAVEL_PURPOSE']], on='TRAVEL_ID', how='left')

    return df_tm
df_tm = map_and_merge_travel_purpose(df_tm, df_tv)

In [26]:
def process_and_map_activity(df_tm, df_acthis, df_code):
    # TRAVEL_ID 기준으로 ACTIVITY_TYPE_CD 리스트 생성
    df_list = df_acthis.groupby('TRAVEL_ID')['ACTIVITY_TYPE_CD'].apply(list).reset_index()
    
    # df_tm에 ACTIVITY_TYPE_CD 병합
    df_tm = pd.merge(df_tm, df_list[['TRAVEL_ID', 'ACTIVITY_TYPE_CD']], on='TRAVEL_ID', how='left')
    
    # TRAVEL_PURPOSE 컬럼을 세미콜론으로 구분하여 리스트로 변환
    df_tm['TRAVEL_PURPOSE'] = df_tm['TRAVEL_PURPOSE'].apply(lambda x: list(map(int, x.split(';'))))
    
    # 가중치를 계산하는 함수 정의
    def calculate_weighted_activity(row):
        activity_values = row['ACTIVITY_TYPE_CD']
        travel_purpose_values = row['TRAVEL_PURPOSE']
        
        # TRAVEL_PURPOSE 값에 없는 항목을 제외하고 필터링
        filtered_values = [value for value in activity_values if value in travel_purpose_values]
        
        # 각 값의 개수를 세고, TRAVEL_PURPOSE에 있는 값에 가중치를 2배 주기
        weighted_values = []
        for value in filtered_values:
            count = activity_values.count(value)  # 해당 값의 발생 횟수
            weight = 2 if value in travel_purpose_values else 1  # TRAVEL_PURPOSE에 있으면 가중치 2배
            weighted_values.append(count * weight)  # 가중치 적용
        
        # 가중치가 가장 큰 값을 찾기
        if weighted_values:
            max_weight = max(weighted_values)
            max_value = filtered_values[weighted_values.index(max_weight)]  # 가중치가 가장 큰 값
            return max_value
        else:
            return 99  # 가중치가 없으면 99 반환

    # 각 행에 대해 계산 후, 'ACTIVITY' 컬럼에 저장
    df_tm['ACTIVITY'] = df_tm.apply(calculate_weighted_activity, axis=1)
    
    # 'ACTIVITY' 값에 대해 코드에서 이름으로 매핑
    df_tm['ACTIVITY'] = df_tm['ACTIVITY'].astype(str)
    df_tm['ACTIVITY'] = df_tm['ACTIVITY'].map(df_code[df_code['cd_a'] == 'ACT'].set_index('cd_b')['cd_nm'])

    return df_tm
df_tm = process_and_map_activity(df_tm, df_acthis, df_code)

In [27]:
def preprocessed(df_visarea, df_acthis, df_code, df_tm):
    # 제외할 VISIT_AREA_TYPE_CD 값 리스트
    exclude_values = [9, 11, 12, 21, 22, 23, 24]
    
    # VISIT_AREA_TYPE_CD 값 제외하고 필터링
    filtered_df = df_visarea[~df_visarea['VISIT_AREA_TYPE_CD'].isin(exclude_values)]
    
    # 필요한 컬럼만 선택
    filtered_df = filtered_df[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 
                               'X_COORD', 'Y_COORD', 'REVISIT_YN', 'DGSTFN', 
                               'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'Calculated_Final_Score', 'SIDO_NM', 'SGG_NM', 'DONG_NM']]
    
    # filtered_df와 df_acthis 병합
    df = pd.merge(
        filtered_df,
        df_acthis[['VISIT_AREA_ID', 'TRAVEL_ID', 'ACTIVITY_TYPE_CD']],
        on=['VISIT_AREA_ID', 'TRAVEL_ID'],
        how='left'
    )
    
    # ACTIVITY_TYPE_CD 결측치 처리 및 형 변환
    df['ACTIVITY_TYPE_CD'].fillna(99, inplace=True)
    df['ACTIVITY_TYPE_CD'] = df['ACTIVITY_TYPE_CD'].astype(int).astype(str)
    
    # ACTIVITY_TYPE_CD 코드 매핑
    activity_map = df_code[df_code['cd_a'] == 'ACT'].set_index('cd_b')['cd_nm']
    df['ACTIVITY_TYPE_CD'] = df['ACTIVITY_TYPE_CD'].map(activity_map)
    
    # merged_df와 df_tm 병합
    df = pd.merge(
        df,
        df_tm[['TRAVEL_ID', 'ACTIVITY']],
        on='TRAVEL_ID',
        how='left'
    )
    
    # REVISIT_YN 값 변환
    df['REVISIT_YN'] = df['REVISIT_YN'].replace({'Y': 1, 'N': 0})
    
    return df

df = preprocessed(df_visarea, df_acthis, df_code, df_tm)

C:\Users\carra\AppData\Local\Temp\ipykernel_13168\1591365335.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ACTIVITY_TYPE_CD'].fillna(99, inplace=True)
C:\Users\carra\AppData\Local\Temp\ipykernel_13168\1591365335.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['REVISIT_YN'] = df['R

### 가중치 계산

In [30]:
def calculate_weights(df, df_cluster):
    # 초기 가중치 계산
    df['ACT_WEIGHT'] = np.where(
        df['ACTIVITY_TYPE_CD'] == df['ACTIVITY'],
        df['DGSTFN'] * 0.4 +
        df['REVISIT_YN'] * 0.3 +
        df['REVISIT_INTENTION'] * 0.15 +
        df['RCMDTN_INTENTION'] * 0.15,
        0
    )
    
    # 최대 가중치 값으로 정규화
    max_weight = df['ACT_WEIGHT'].max()
    df['ACT_WEIGHT'] = df['ACT_WEIGHT'] / max_weight
    
    # 총 가중치 계산
    df['TOTAL_WEIGHT'] = df['Calculated_Final_Score'] + df['ACT_WEIGHT']
    
    # 클러스터 데이터 병합
    df = pd.merge(df, df_cluster[['TRAVEL_ID', 'Cluster']], on='TRAVEL_ID', how='left')
    
    # 필요한 컬럼만 선택
    df = df[['TRAVEL_ID', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 
                           'X_COORD', 'Y_COORD', 'SIDO_NM', 'SGG_NM', 'DONG_NM', 'TOTAL_WEIGHT', 'Cluster']]
    
    return df

df = calculate_weights(df, df_cluster)

In [31]:
df.head()

,TRAVEL_ID,VISIT_AREA_NM,ROAD_NM_ADDR,X_COORD,Y_COORD,SIDO_NM,SGG_NM,DONG_NM,TOTAL_WEIGHT,Cluster
0,e_e000004,화성 관광열차 안내소 연무대 매표소,경기 수원시 팔달구 창룡대로103번길 20,127.023339,37.287878,경기,수원시 팔달구,매향동,0.800000,0
1,e_e000004,창룡문,경기 수원시 팔달구 남수동 152,127.025143,37.287791,경기,수원시 팔달구,남수동,1.536842,0
2,e_e000006,경춘선 자전거길,경기 가평군 청평면 하천리 172-2,127.436200,37.745958,경기,가평군,청평면 하천리,1.420000,1
3,e_e000009,농협안성팜랜드,경기 안성시 공도읍 대신두길 28,127.193517,36.991317,경기,안성시,공도읍 신두리,0.640000,1
4,e_e000009,농협안성팜랜드,경기 안성시 공도읍 대신두길 28,127.193517,36.991317,경기,안성시,공도읍 신두리,1.376842,1


In [79]:
def haversine(lat1, lon1, lat2, lon2): 
    # 지구의 반지름 (킬로미터 단위)
    R = 6371.0
    
    # 위도와 경도를 라디안 단위로 변환
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # 위도와 경도의 차이 계산
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # 하버사인 공식 적용
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 거리 계산
    distance = R * c
    return distance
from geopy.distance import geodesic

def activity_second_rmd(df, cluster_label, user_lat, user_lon, radius=3, top_n=10):

    # 클러스터 집단 필터링
    cluster_data = df[df['Cluster'] == cluster_label].copy()
    
    # 거리 계산
    cluster_data['DISTANCE'] = cluster_data.apply(
        lambda row: geodesic((user_lat, user_lon), (row['Y_COORD'], row['X_COORD'])).km,
        axis=1
    )
    
    # 반경 내 데이터 필터링
    nearby_data = cluster_data[cluster_data['DISTANCE'] <= radius]
    
    # X_COORD와 Y_COORD 기준으로 만족도 평균 계산
    grouped_data = (
        nearby_data.groupby(['X_COORD', 'Y_COORD'], as_index=False)
        .agg({'TOTAL_WEIGHT': 'mean'})
        .rename(columns={'TOTAL_WEIGHT': 'TOTAL_WEIGHT_avg'})
    )
    
    # 그룹화된 데이터와 원래 데이터 병합하여 모든 컬럼 유지
    merged_data = pd.merge(nearby_data, grouped_data, on=['X_COORD', 'Y_COORD'], how='left')
    
    # 만족도 기준 상위 데이터 선택
    top_recommendations = (
        merged_data.sort_values(by='TOTAL_WEIGHT_avg', ascending=False)
        .drop_duplicates(subset=['X_COORD', 'Y_COORD'])  # 중복 제거
        .head(top_n)
        .reset_index(drop=True)
    )
    
    # 결과 반환: 원래 컬럼 + 평균 만족도 및 거리 포함
    return top_recommendations[['VISIT_AREA_NM', 'ROAD_NM_ADDR', 'X_COORD', 'Y_COORD', 'TOTAL_WEIGHT_avg', 'DISTANCE', 'Cluster']]



In [77]:
from geopy.distance import geodesic

def activity_first_rmd(df, cluster_label, user_lat, user_lon, top_n=10):
    # 클러스터 집단 필터링
    cluster_data = df[df['Cluster'] == cluster_label].copy()
    
    # X_COORD와 Y_COORD 기준으로 만족도 평균 계산
    grouped_data = (
        cluster_data.groupby(['X_COORD', 'Y_COORD'], as_index=False)
        .agg({'TOTAL_WEIGHT': 'mean'})
        .rename(columns={'TOTAL_WEIGHT': 'TOTAL_WEIGHT_avg'})
    )
    
    # 그룹화 결과를 원래 데이터에 병합하여 모든 컬럼 유지
    merged_data = pd.merge(cluster_data, grouped_data, on=['X_COORD', 'Y_COORD'], how='left')
    
    # 만족도 기준으로 정렬하여 상위 N개 선택
    top_recommendations = (
        merged_data.sort_values(by='TOTAL_WEIGHT_avg', ascending=False)
        .drop_duplicates(subset=['X_COORD', 'Y_COORD'])  # 중복 제거
        .head(top_n)
        .reset_index(drop=True)
    )
    
    # 거리 계산
    def calculate_distance(row):
        activity_coords = (row['Y_COORD'], row['X_COORD'])  # 액티비티 좌표
        user_coords = (user_lat, user_lon)  # 사용자 좌표
        return geodesic(user_coords, activity_coords).km

    top_recommendations['distance_to_user'] = top_recommendations.apply(calculate_distance, axis=1)
    top_recommendations['distance_to_user'] = top_recommendations['distance_to_user'].round(2)
    
    # 반환 데이터: 추천 액티비티 이름, 만족도, 거리
    return top_recommendations[['VISIT_AREA_NM', 'ROAD_NM_ADDR', 'X_COORD', 'Y_COORD', 'TOTAL_WEIGHT_avg', 'distance_to_user', 'Cluster']]


In [69]:
def des_act_rmd(df, cluster_label, target_sido, target_sgg=None, target_dong=None, top_n=10):
    # 클러스터 필터링
    filtered_df = df[df['Cluster'] == cluster_label]
    
    if not target_sgg and not target_dong:
        filtered_df = filtered_df[filtered_df['SIDO_NM'].str.contains(target_sido, na=False)]
    elif not target_dong:
        filtered_df = filtered_df[
            filtered_df['SIDO_NM'].str.contains(target_sido, na=False) &
            filtered_df['SGG_NM'].str.contains(target_sgg, na=False)
        ]
    else:
        filtered_df = filtered_df[
            filtered_df['SIDO_NM'].str.contains(target_sido, na=False) &
            filtered_df['SGG_NM'].str.contains(target_sgg, na=False) &
            filtered_df['DONG_NM'].str.contains(target_dong, na=False)
        ]

    
    # X_COORD와 Y_COORD 기준 그룹화 및 평균 계산
    grouped_df = (
        filtered_df.groupby(['X_COORD', 'Y_COORD'], as_index=False)
        .agg({'TOTAL_WEIGHT': 'mean'})
    )
    
    # 그룹화 결과를 기존 df와 병합하여 다른 컬럼 유지
    merged_df = pd.merge(filtered_df, grouped_df, on=['X_COORD', 'Y_COORD'], suffixes=('', '_avg'))

    # 중복 제거 및 상위 N개 추천
    unique_activities = merged_df.drop_duplicates(subset=['X_COORD', 'Y_COORD'])
    top_activities = unique_activities.nlargest(top_n, 'TOTAL_WEIGHT_avg')
    top_activities = top_activities.reset_index(drop=True)

    # 결과 반환 (모든 컬럼 포함)
    return top_activities[['VISIT_AREA_NM', 'ROAD_NM_ADDR', 'X_COORD', 'Y_COORD', 'TOTAL_WEIGHT_avg', 'Cluster']]

In [91]:
# 예시 사용
target_sido = '부산'  # '용산'만 입력해도 '용산구'에 대한 정보가 검색됨
target_sgg = '사하'
target_dong = ''  # 빈 문자열로 전달
cluster_label = 1
result_sido = des_act_rmd(df, cluster_label, target_sido, target_sgg, target_dong, top_n=10)
result_sido


,VISIT_AREA_NM,ROAD_NM_ADDR,X_COORD,Y_COORD,TOTAL_WEIGHT_avg,Cluster
0,몰운대,부산 사하구 다대동 산 144,128.970952,35.037821,2.920000,1
1,힐링 서프,부산 사하구 다대로 692,128.965873,35.048447,2.151053,1
2,다대포해수욕장,부산 사하구 다대동,128.962741,35.046526,1.613985,1
3,을숙도 생태공원,부산 사하구 하단동 1142,128.948878,35.116073,1.533333,1
4,낙동강하구에코센터,부산 사하구 낙동남로 1240,128.945830,35.104391,1.410526,1
5,부산 현대미술관,부산 사하구 낙동남로 1191,128.942723,35.109251,1.200000,1
6,감천문화마을,부산 사하구 감내1로 200,129.009428,35.096300,0.786711,1


In [78]:
# 사용자 정보
user_lat = 37.567  # 사용자 위도
user_lon = 126.978  # 사용자 경도
cluster_label = 1  # 사용자가 속한 클러스터 번호

# 추천 함수 실행
top_activities = activity_first_rmd(df, cluster_label, user_lat, user_lon, top_n=10)

top_activities

,VISIT_AREA_NM,ROAD_NM_ADDR,X_COORD,Y_COORD,TOTAL_WEIGHT_avg,distance_to_user,Cluster
0,링크 아트센터,서울 종로구 대학로14길 29,127.002296,37.584689,2.936667,2.91,1
1,몰운대,부산 사하구 다대동 산 144,128.970952,35.037821,2.920000,332.85,1
2,변산 마실 길 2코스(노루목 상사와 길),전북특별자치도 부안군 변산면 운산리 531-12,126.518035,35.668920,2.811053,214.61,1
3,경주계림,경북 경주시 교동 1,129.218916,35.832575,2.800000,277.73,1
4,새롬 맛집,강원 속초시 중앙시장로6길 28,128.589831,38.204465,2.720000,158.46,1
5,화순농협 하나로마트,전남 화순군 화순읍 쌍충로 74,126.982207,35.058763,2.690000,278.33,1
6,통 연달아 공원,경남 통영시 산양읍 산양일주로 1115,128.399268,34.769164,2.681053,335.75,1
7,퍼플 모스,제주특별자치도 제주시 애월읍 하귀로 53,126.402815,33.480489,2.670000,456.39,1
8,주파크,경기 포천시 소흘읍 죽엽산로 645,127.165341,37.777869,2.660000,28.65,1
9,에버랜드 장미축제,경기 용인시 처인구 포곡읍 에버랜드로 199,127.198405,37.291037,2.660000,36.31,1


In [80]:
# 사용자 정보
user_lat = 37.567  # 사용자 위도
user_lon = 126.978  # 사용자 경도
cluster_label = 1  # 사용자가 속한 클러스터 번호

# 추천 함수 실행
top_activities = activity_second_rmd(df, cluster_label, user_lat, user_lon, top_n=10)

top_activities

,VISIT_AREA_NM,ROAD_NM_ADDR,X_COORD,Y_COORD,TOTAL_WEIGHT_avg,DISTANCE,Cluster
0,링크 아트센터,서울 종로구 대학로14길 29,127.002296,37.584689,2.936667,2.908744,1
1,국립현대미술관 서울관,서울 종로구 소격동 165-10,126.980555,37.578635,2.571053,1.310865,1
2,세종문화회관 대극장,서울 종로구 세종대로 175,126.976070,37.572862,2.551053,0.672614,1
3,익선동 한옥거리,서울 종로구 익선동,126.989694,37.574335,2.422105,1.315271,1
4,크레도 종로 프리미엄점,서울 종로구 종로 120,126.990844,37.570075,2.380000,1.184872,1
5,안중근 의사 기념관,서울 중구 소월로 91,126.980408,37.553524,2.251053,1.510732,1
6,해방촌 예술마을,서울 용산구 용산동2가 1-457,126.981727,37.546255,2.231053,2.325890,1
7,올리브영 서울역점,서울 중구 한강대로 405,126.971129,37.555867,2.200000,1.376745,1
8,그랑 핸드 남산,서울 용산구 두텁바위로60길 49,126.983555,37.548922,2.200000,2.065585,1
9,108하늘계단,서울 용산구 후암동 415-3,126.981890,37.546446,2.200000,2.306997,1
